In [3]:
pip install earthengine-api

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.8/354.8 kB 6.8 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
import xarray as xr
import rioxarray as rxr
import numpy as np
# from Py6S import *
import datetime
import math
import os
import sys
import atmos
from datetime import datetime, timedelta
import act
import ee

ModuleNotFoundError: No module named 'act'

In [10]:
ee.Authenticate()

True

In [11]:
ee.Initialize()

In [ ]:
data = rxr.open_rasterio('../Ks/Clipped_car1.tif')

In [12]:
# Given solar radiance values
solar_radiance = {
    'B1': 196.780,
    'B2': 182.010,
    'B3': 157.540,
    'B4': 110.860
}

# Hypothetical distance from the sun (in astronomical units)
distance_to_sun_au = 1.0

# Calculate irradiance for each band
irradiance_values = {}
for band, radiance in solar_radiance.items():
    irradiance = radiance / (distance_to_sun_au ** 2)
    irradiance_values[band] = irradiance

# Print the resulting dictionary
print(irradiance_values)


{'B1': 196.78, 'B2': 182.01, 'B3': 157.54, 'B4': 110.86}


In [13]:
# top of atmosphere reflectance
toa = data/2047

NameError: name 'data' is not defined

In [17]:
scene_date = datetime.strptime('05-Mar-2022 04:38:32:4464', "%d-%b-%Y %H:%M:%S:%f")
solar_z = 191.21318
geom = ee.Geometry.Polygon(
    [[[77.525964, 13.163993],
      [77.683703, 13.133772],
      [77.651635, 12.973449],
      [77.493987, 13.003631]]]
)

In [16]:
scene_date

datetime.datetime(2022, 3, 5, 4, 38, 32, 446400)

In [18]:
def round_date(date, xhour):
    """
    Rounds a date to the closest 'x' hours
    """
    rounded_date = date - timedelta(hours=date.hour % xhour, minutes=date.minute, seconds=date.second, microseconds=date.microsecond)
    rounded_date += timedelta(hours=xhour//2)
    return rounded_date

In [19]:
rounded_date = round_date(scene_date, 6)
print(rounded_date)


2022-03-05 03:00:00


In [23]:
def round_month(date):
    """
    Round date to the closest month
    """
#     date = ee.Date(date)
    # Start of THIS month
    m1 = ee.Date.fromYMD(date.get('year'), date.get('month'), ee.Number(1))
    
    # Start of NEXT month
    m2 = m1.advance(1, 'month')
      
    # Difference from date
    d1 = ee.Number(date.difference(m1, 'day')).abs()
    d2 = ee.Number(date.difference(m2, 'day')).abs()
    
    # Return closest start of month
    closest_month = ee.Algorithms.If(d2.gt(d1), m1, m2)
    
    return ee.Date(closest_month)


In [24]:
import ee

def water(geom, date):
    """
    Water vapour column above target at time of image aquisition.
    
    (Kalnay et al., 1996, The NCEP/NCAR 40-Year Reanalysis Project. Bull. 
    Amer. Meteor. Soc., 77, 437-471)
    """
    
    # Point geometry required
    centroid = geom.centroid()
    
    # Convert Python datetime to Earth Engine ee.Date
    H2O_date = ee.Date(round_date(date, 6))
    
    # Filtered water collection
    water_ic = ee.ImageCollection('NCEP_RE/surface_wv').filterDate(H2O_date, H2O_date.advance(1, 'month'))
    
    # Water image
    water_img = ee.Image(water_ic.first())
    
    # Water vapour at target
    water = water_img.reduceRegion(reducer=ee.Reducer.mean(), geometry=centroid).get('pr_wtr')
                                        
    # Convert to Py6S units (Google = kg/m^2, Py6S = g/cm^2)
    water_Py6S_units = ee.Number(water).divide(10)   
    
    # Get the numerical result
    water_value = water_Py6S_units.getInfo()
    
    return water_value


In [25]:
water(geom,scene_date)

1.8200000762939452

In [26]:
import ee

def ozone(geom, date):
    """
    Returns ozone measurement from the merged TOMS/OMI dataset
    or uses the fill value (mean value for that latlon and day-of-year).
    """
    
    # Point geometry required
    centroid = geom.centroid()
       
    def ozone_measurement(centroid, o3_date):
        # Filtered ozone collection
        
        ozone_ic = ee.ImageCollection('TOMS/MERGED').filterDate(o3_date, o3_date.advance(1, 'month'))
        
        # Ozone image
        ozone_img = ee.Image(ozone_ic.first())
        
        # Ozone value if TOMS/OMI image exists, else use fill value
        ozone = ee.Algorithms.If(
            ozone_img,
            ozone_img.reduceRegion(reducer=ee.Reducer.mean(), geometry=centroid).get('ozone'),
            ozone_fill(centroid, o3_date)
        )
        
        return ozone
      
    def ozone_fill(centroid, o3_date):
        """
        Gets our ozone fill value (i.e. mean value for that doy and latlon)
        """
        # Ozone fills (i.e., one band per doy)
        
        ozone_fills = ee.ImageCollection('users/samsammurphy/public/ozone_fill').toList(366)
        
        # Day of year index
        jan01 = ee.Date.fromYMD(o3_date.get('year'), 1, 1)
        doy_index = o3_date.difference(jan01, 'day').toInt()  # (NB. index is one less than doy, so no need to +1)
        
        # Day of year image
        fill_image = ee.Image(ozone_fills.get(doy_index))
        
        # Return scalar fill value
        return fill_image.reduceRegion(reducer=ee.Reducer.mean(), geometry=centroid).get('ozone')
     
    # O3 datetime in 24-hour intervals
    o3_date = ee.Date(round_date(date, 24))
    
    # TOMS temporal gap
    toms_gap = ee.DateRange('2022-01-01', '2022-09-01')  
    
    # Avoid TOMS gap entirely
    ozone = ee.Algorithms.If(toms_gap.contains(o3_date),
                             ozone_fill(centroid, o3_date),
                             ozone_measurement(centroid, o3_date))
    
    # Fix other data gaps (e.g., spatial, missing images, etc.)
    ozone = ee.Algorithms.If(ozone, ozone, ozone_fill(centroid, o3_date))
    
    # Convert to Py6S units 
    ozone_py6s_units = ee.Number(ozone).divide(1000)  # Dobson units are milli-atm-cm
    
    ozone_value = ozone_py6s_units.getInfo()
    
    return ozone_value


In [27]:
ozone(geom,scene_date)

0.24941249999999998

In [28]:
def aerosol(geom, date):
    """
    Aerosol Optical Thickness.

    try:
        MODIS Aerosol Product (monthly)
    except:
        fill value
    """
    date = ee.Date(date)
    
    def aerosol_fill(date):
        """
        MODIS AOT fill value for this month (i.e., no data gaps)
        """
        return (
            ee.Image('users/samsammurphy/public/AOT_stack')
            .select([ee.String('AOT_').cat(date.format('M'))])
            .rename(['AOT_550'])
        )

    def aerosol_this_month(date):
        """
        MODIS AOT original data product for this month (i.e., some data gaps)
        """
        # Image for this month
        img = ee.ImageCollection('MODIS/006/MOD08_M3')\
            .filterDate(round_month(date))\
            .first()

        # Fill missing month (?)
        img = ee.Algorithms.If(
            img,
            # All good
            img.select(['Aerosol_Optical_Depth_Land_Mean_Mean_550'])
                .divide(1000)
                .rename(['AOT_550']),
            # Missing month
            aerosol_fill(date)
        )

        return img

    def get_AOT(AOT_band, geom):
        """
        AOT scalar value for the target
        """
        return (
            ee.Image(AOT_band)
            .reduceRegion(reducer=ee.Reducer.mean(), geometry=geom.centroid())
            .get('AOT_550')
        )

    after_modis_start = date.difference(ee.Date('2022-05-03'), 'month').gt(0)

    AOT_band = ee.Algorithms.If(
        after_modis_start,
        aerosol_this_month(date),
        aerosol_fill(date)
    )

    AOT = get_AOT(AOT_band, geom)

    AOT = ee.Algorithms.If(
        AOT,
        AOT,
        get_AOT(aerosol_fill(date), geom)
    )
    # i.e. check reduce region worked (else force fill value)
    AOT = AOT.getInfo()
    return AOT

In [29]:
aerosol(geom,scene_date)

/opt/conda/lib/python3.10/site-packages/ee/deprecation.py:202: DeprecationWarning: 

Attention required for MODIS/006/MOD08_M3! You are using a deprecated asset.
To ensure continued functionality, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MOD08_M3

  warnings.warn(warning, category=DeprecationWarning)


0.17933334410190582

In [30]:
h2o = water(geom,scene_date)
o3 = ozone(geom,scene_date)
aot = aerosol(geom,scene_date)

In [27]:
SRTM = ee.Image('CGIAR/SRTM90_V4')# Shuttle Radar Topography mission covers *most* of the Earth
alt = SRTM.reduceRegion(reducer = ee.Reducer.mean(),geometry = geom.centroid()).get('elevation').getInfo()
km = alt/1000

In [6]:
Wavelength(0.43, 0.50)
def spectralResponseFunction(bandname):
    """
    Extract spectral response function for given band name
    """
    bandSelect = {
        'B1':Wavelength(0.45, 0.52),
        'B2':Wavelength(0.52, 0.59),
        'B3':Wavelength(0.62, 0.68),
        'B4':Wavelength(0.77, 0.86)
        }
    return bandSelect[bandname]

NameError: name 'Wavelength' is not defined

In [28]:
# Instantiate
s = SixS()

# Atmospheric constituents
s.atmos_profile = AtmosProfile.UserWaterAndOzone(h2o,o3)
s.aero_profile = AeroProfile.Continental
s.aot550 = aot

# Earth-Sun-satellite geometry
s.geometry = Geometry.User()
s.geometry.view_z = 0               # always NADIR (I think..)
s.geometry.solar_z = solar_z        # solar zenith angle
s.geometry.month = scene_date.month # month and day used for Earth-Sun distance
s.geometry.day = scene_date.day     # month and day used for Earth-Sun distance
s.altitudes.set_sensor_satellite_level()
s.altitudes.set_target_custom_altitude(km)

In [30]:
def toa_to_rad(bandname):
    """
    Converts top of atmosphere reflectance to at-sensor radiance
    """
    
    # solar exoatmospheric spectral irradiance
    ESUN = irradiance_values[bandname]
    solar_angle_correction = math.cos(math.radians(solar_z))
    
    # Earth-Sun distance (from day of year)
    doy = scene_date.timetuple().tm_yday
    d = 1 - 0.01672 * math.cos(0.9856 * (doy-4))# http://physics.stackexchange.com/questions/177949/earth-sun-distance-on-a-given-day-of-the-year
   
    # conversion factor
    multiplier = ESUN*solar_angle_correction/(math.pi*d**2)

    # at-sensor radiance
    rad = toa.select(bandname).multiply(multiplier)
    
    return rad

In [31]:
def surface_reflectance(bandname):
    """
    Calculate surface reflectance from at-sensor radiance given waveband name
    """
    
    # run 6S for this waveband
    s.wavelength = spectralResponseFunction(bandname)
    s.run()
    
    # extract 6S outputs
    Edir = s.outputs.direct_solar_irradiance             #direct solar irradiance
    Edif = s.outputs.diffuse_solar_irradiance            #diffuse solar irradiance
    Lp   = s.outputs.atmospheric_intrinsic_radiance      #path radiance
    absorb  = s.outputs.trans['global_gas'].upward       #absorption transmissivity
    scatter = s.outputs.trans['total_scattering'].upward #scattering transmissivity
    tau2 = absorb*scatter                                #total transmissivity
    
    # radiance to surface reflectance
    rad = toa_to_rad(bandname)
    ref = rad.subtract(Lp).multiply(math.pi).divide(tau2*(Edir+Edif))
    
    return ref


In [38]:
# surface reflectance rgb
b = surface_reflectance('B1')
g = surface_reflectance('B2')
r = surface_reflectance('B3')
ref = r.addBands(g).addBands(b)


ExecutionError: 6S executable not found.

In [3]:
import Py6S
Py6S.sixs_path = '/home/jovyan/Ks/Py6S-1.9.2/Py6S'

In [17]:
import os
new_directory = "/home/jovyan/"
os.chdir(new_directory)

In [18]:
!pwd

/home/jovyan


In [5]:
os.path.exists("/home/jovyan/Ks/")


True

In [19]:
from Py6S import *
# Py6S.sixs_path = '/home/jovyan/Ks/Py6S-1.9.2/Py6S'
SixS.test()
s = SixS()
s.run()

6S wrapper script by Robin Wilson
Error: cannot find the sixs executable in $PATH or current directory.


ExecutionError: 6S executable not found.

In [10]:
!pwd

/home/jovyan/lulc-notebook-workspace/Ks
